In [1]:
import numpy as np
from Object_Detection import ObjectDectection
from yolo_helper import GeneratorMultipleOutputs

Using TensorFlow backend.


In [2]:
reduced_dataset_folder = '/home/usuario/repos/Object_Localization/challenge_dataset_v3'
train_folder = reduced_dataset_folder + '/images/train'
val_folder = reduced_dataset_folder + '/images/val'
annotations_dict_train = np.load(reduced_dataset_folder+'/annotations_train.npy').flat[0]
annotations_dict_val = np.load(reduced_dataset_folder+'/annotations_val.npy').flat[0]
classes = list(annotations_dict_train.keys())
print(classes)

['n02165456', 'n02226429', 'n01644373', 'n02279972', 'n01632458', 'n02256656', 'n01774384', 'n01770393']


In [3]:
NUMBER_OF_CLASSES = 8
IMAGE_H = 32
IMAGE_W = 32
GRID_H = IMAGE_H // 32
GRID_W = IMAGE_W // 32
NUMBER_OF_BBOXES = 1
object_detection = ObjectDectection(NUMBER_OF_CLASSES, IMAGE_H, IMAGE_W, NUMBER_OF_BBOXES)

In [19]:
batch_size = 60
train_generator = GeneratorMultipleOutputs(annotations_dict_train, 
                            train_folder, batch_size, classes = classes,
                            target_size=(IMAGE_H,IMAGE_W),
                            GRID_H = GRID_H,
                            GRID_W = GRID_W, 
                            flip_horizontal = True, 
                            flip_vertical = True,
                            crop_per=0.2
                           )

Found 2400 images belonging to 8 classes.


In [20]:
val_generator = GeneratorMultipleOutputs(annotations_dict_val, 
                            val_folder, batch_size, classes = classes,
                            target_size=(IMAGE_H,IMAGE_W),
                            GRID_H = GRID_H,
                            GRID_W = GRID_W,
                            flip_horizontal = True, 
                            flip_vertical= True,
                            crop_per=0.2
                           )

Found 480 images belonging to 8 classes.


In [21]:
import tensorflow as tf
import keras.backend as K

start_bbox_idx = 1 + len(classes)

def bounding_box_mse(y_true, y_pred):
    indexes = tf.where(K.equal(y_true[:,:,:,:,0], K.ones_like(y_true[:,:,:,:,0])))
    y_true_pos = tf.gather_nd(y_true, indexes)[:,start_bbox_idx:start_bbox_idx+4]
    y_pred_pos = tf.gather_nd(y_pred, indexes)[:,start_bbox_idx:start_bbox_idx+4]
    return K.mean(K.square(y_pred_pos - y_true_pos), axis=-1)

        
    
def categorical_cross_entropy_loss(y_true, y_pred):
    indexes = tf.where(K.equal(y_true[:,:,:,:,0], K.ones_like(y_true[:,:,:,:,0])))
    y_true_pos = tf.gather_nd(y_true, indexes)[:, start_classes_idx:start_classes_idx+NUMBER_OF_CLASSES]
    y_pred_pos = tf.gather_nd(y_pred, indexes)[:, start_classes_idx:start_classes_idx+NUMBER_OF_CLASSES]
    return K.mean(K.categorical_crossentropy(y_true_pos, K.softmax(y_pred_pos)))
        
    

def object_bin_cross_entropy_loss(y_true, y_pred):
    indexes = tf.where(K.equal(y_true[:,:,:,:,0], K.ones_like(y_true[:,:,:,:,0])))
    y_true_pos = tf.gather_nd(y_true, indexes)
    y_pred_pos = tf.gather_nd(y_pred, indexes)
    return K.mean(K.mean(K.binary_crossentropy(y_true_pos[:,:1], K.sigmoid(y_pred_pos[:,:1])), axis=-1))
    
    
def no_object_bin_cross_entropy_loss(y_true, y_pred):
    indexes_neg = tf.where(K.equal(y_true[:,:,:,:,0], K.zeros_like(y_true[:,:,:,:,0])))
    y_true_pos = tf.gather_nd(y_true, indexes_neg)
    y_pred_pos = tf.gather_nd(y_pred, indexes_neg)
    return K.mean(K.mean(K.binary_crossentropy(y_true_pos[:,:1], K.sigmoid(y_pred_pos[:,:1])), axis=-1))
        

In [22]:
image, y_true = next(val_generator)

In [23]:
y_pred = object_detection.model.predict_on_batch(image)

In [24]:
out_0 = bounding_box_mse(tf.constant(y_true, dtype='float32'), tf.constant(y_pred, dtype='float32'))
#out_1 = focal_loss_2(tf.constant(y_true, dtype='float32'), tf.constant(y_pred, dtype='float32'))
#out_2 = focal_loss_3(tf.constant(y_true, dtype='float32'), tf.constant(y_pred, dtype='float32'))

In [25]:
sess = tf.Session()
with sess.as_default():
    print(out_0.eval().shape)
    
# 0.086642854
# 0.08664659

(60,)


In [201]:
y_true

array([[[[[1.]]]]])

In [205]:
y_true, y_pred

(array([[[[[1.]]]]]), array([[[[[0.9]]]]], dtype=float32))

In [105]:
def cross_entropy(predictions, targets, epsilon=1e-12):
    """
    Computes cross entropy between targets (encoded as one-hot vectors)
    and predictions. 
    Input: predictions (N, k) ndarray
           targets (N, k) ndarray        
    Returns: scalar
    """
    predictions = np.clip(predictions, epsilon, 1. - epsilon)
    N = predictions.shape[0]
    ce = -np.sum(targets*np.log(predictions+1e-9))/N
    return ce

predictions = np.array([[0.25]])
targets = np.array([[1]])
x = cross_entropy(predictions, targets)
print(x)

1.3862943571198905
